### input data 형태 설정 & 모델 설정

In [ ]:
input_shape = x_train_3d.shape  # (batch_size, time_steps, num_features)
model = create_lstm_normal_model(input_shape)

### model 학습

In [ ]:
history = model.fit(x_train_3d, y_train_3d, 
                    epochs=42, 
                    validation_data=(x_test_3d, y_test_3d))

### 예측값 출력

In [ ]:
y_pred = model.predict(x_test_3d)

### MSE 출력

In [ ]:
test_results = model.evaluate(x_test_3d, y_test_3d)
test_mse = test_results[0]
print("\nTest MSE:", test_mse)

### Hyperparameter Tuning

In [ ]:
from keras.models import Model
from keras.layers import LSTM, Dense, Input, Reshape
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
from keras.losses import MeanSquaredError
import tensorflow as tf
import numpy as np

def create_lstm_normal_model(units1, units2, activation, learning_rate):
    # Input Layer
    inputs = Input(shape=(input_shape[1], input_shape[2]))  # (time_step, num_features)
    lstm_out = LSTM(units=units1, return_sequences=True)(inputs)  # first LSTM Layer
    lstm_out = LSTM(units=units1, return_sequences=False)(lstm_out)  # second LSTM Layer
    # Dense Layer
    dense_out = Dense(units2, activation=activation)(lstm_out)  # Dense 레이어에 activation 함수 적용
    # final Output Layer
    outputs = Dense(2)(dense_out)  # 2차원 출력 (ex: (batch_size, 2))
    outputs = keras.layers.Reshape((1, 2))(outputs)
    model = keras.models.Model(inputs = inputs, outputs = outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate = learning_rate),
        loss = 'mean_squared_error',
        metrics = [
            keras.metrics.MeanSquaredError(),
            keras.metrics.RootMeanSquaredError(),
            keras.metrics.MeanAbsoluteError()
        ]
    )
    return model

# KerasRegressor 설정
input_shape = x_train_3d.shape  # (batch_size, time_steps, num_features)
lstm_regressor = KerasRegressor(model=create_lstm_normal_model, verbose=3)

# 파라미터 범위 설정
param_grid = {
    'model__units1': [32, 128],
    'model__units2': [16, 32, 64],
    'model__activation': [None, 'relu'],
    'model__learning_rate': [0.001],
    'epochs' : [42, 44, 46, 48, 50]
}

# GridSearchCV 설정
grid_search = GridSearchCV(estimator=lstm_regressor, param_grid=param_grid)
grid_result = grid_search.fit(x_train_3d, y_train_reshaped)  # 2차원 y_train_reshaped 사용 ▶ GridsearchCV: 2차원의 입력만 받기 가능

# 상위 5개의 결과 출력 및 테스트 MSE 계산
def print_top_5_results(grid_result, x_test_3d, y_test_3d):
    # GridSearch 결과에서 상위 5개 추출
    results = grid_result.cv_results_
    
    # 'rank_test_score'를 기준으로 정렬하여 상위 5개 인덱스 추출
    top_5_idx = np.argsort(results['rank_test_score'])[:5]
    
    # 상위 5개 결과 출력
    print("\nTop 5 Results:")
    for i, idx in enumerate(top_5_idx):
        print(f"Rank {i+1}")
        print(f"Params: {results['params'][idx]}")
        print(f"Mean Cross-Validation Score (MSE): {results['mean_test_score'][idx]}")
    
    # 각 상위 5개의 파라미터로 모델을 학습한 후 테스트 데이터에 대해 MSE 계산
    print("\nTop 5 Test MSE:")
    mse_object = MeanSquaredError()  # MSE 계산을 위한 객체
    for i, idx in enumerate(top_5_idx):
        best_params = results['params'][idx]
        print(f"\nRank {i+1} - Params: {best_params}")
        
        # 'model__' 접두어 제거
        clean_params = {key.replace('model__', ''): value for key, value in best_params.items()}
        
        # 해당 파라미터로 모델 생성 및 학습
        best_epochs = best_params.pop('epochs')
        model = create_lstm_normal_model(**clean_params)
        model.fit(x_train_3d, y_train_reshaped, epochs=best_epochs, verbose=0)
        
        # 테스트 데이터에 대한 예측 및 MSE 계산 (3차원 y_test_3d 사용)
        y_test_pred = model.predict(x_test_3d)
        
        # MSE 계산
        test_mse = mse_object(y_test_3d.reshape(-1, 2), y_test_pred.reshape(-1, 2)).numpy()  # MSE 객체로 계산
        print(f"Test MSE: {test_mse}")

# 함수 실행
print_top_5_results(grid_result, x_test_3d, y_test_3d)